In [67]:
import numpy as np
from enum import Enum
import matplotlib.pyplot as plt
import random
from scipy import stats
import plotly.graph_objects as go

In [68]:
class SYSTEM_TYPE(Enum):
    PRE_SELECTED =0
    SHORTEST_QUEUE = 1
    GENERAL_QUEUE = 2
    DYNAMIC_SERVERS = 3


arrival_xk = np.arange(4) + 1
arrival_pk = (0.25, 0.4, 0.2, 0.15)
custom_arrival = stats.rv_discrete(name='custom_arrival', values=(arrival_xk, arrival_pk))
def get_service_time(server):
    service_xk = np.arange(4) + 5
    service_pk = (0.3, 0.28, 0.25, 0.17)
    service = stats.rv_discrete(name='service_' + str(server), values=(service_xk, service_pk))
    return service.rvs()

class CustomerStat:
    def __init__(self):
        self.id = -1
        self.arrival_time = -1
        self.service_time = -1
        self.inter_arrival_time = 0
        self.service_begins = -1
        self.waiting_time_in_queue = 0
        self.service_ends = -1
        self.time_in_system = -1
        self.idle_time_of_server = 0
        self.server = 'none'

class EventsQueue:
    def __init__(self, devs):
        self.global_time = 0
        self.events = []
        self.devs = devs
    @property
    def size(self):
        return len(self.events)

    def add_event(self, new_event):
        count = self.size
        if count == 0 or new_event.e_time >= self.events[count-1].e_time:
            self.events.append(new_event)
            return 0

        for i in range(0, count - 1):
            if self.events[i].e_time <= new_event.e_time < self.events[i + 1].e_time:
                self.events.insert(i + 1, new_event)
                return 0

    def process_next_event(self):
        if self.size == 0:
            return 0
        self.events[0].execute(self.devs)
        self.global_time = self.events[0].e_time
        del self.events[0]

servers_minimum = 3
servers_maximum = 6
class DEVS:
    def __init__(self):
        self.events_queue = EventsQueue(self)
        self.GLOBAL_TIME = 0.0
        self.customer_queue = []
        self.stats = []
        self.newId = 0
        self.server_idle = {}
        self.last_served_time = {}
        self.servers_count = servers_minimum

        for i in range(self.servers_count):
            self.server_idle[i + 1] = True
            self.last_served_time[i + 1] = 0.0

    def process_next_event(self):
        self.events_queue.process_next_event()
        self.GLOBAL_TIME = self.events_queue.global_time

    def get_preselected_server(self):
        return random.randint(1, self.servers_count)

    def get_shortest_queue_server(self):
       no_queue_servers =  [server_name for server_name, server_is_free in self.server_idle.items() if server_is_free]
       if no_queue_servers:
         return no_queue_servers[0]
       servers = [server_name for server_name, server_is_free in self.server_idle.items()]
       # print(servers)
       shortest = servers[random.randint(0, len(servers) - 1)]
       events = []
       for event in self.events_queue.events:
           if isinstance(event, ServiceEvent):
               events.append(event)
       for server in servers:
           if len(list(filter(lambda x: x.server == server,self.customer_queue))) < len(list(filter(lambda x: x.server == shortest,self.customer_queue))):
               shortest = server
       return shortest


    def get_next_server(self):
        servers = [server_name for server_name, server_is_free in self.server_idle.items() if server_is_free]
        if not servers:
            return None
        random_free_server = servers[random.randint(0, len(servers) - 1)]
        return random_free_server

    def get_dynamic_server(self):
        # print(len(self.customer_queue))
        if len(self.customer_queue) >2 and self.servers_count < servers_maximum:
            self.add_server()
        if len(self.customer_queue) < 1 and self.servers_count > servers_minimum:
            self.remove_server()
        return self.get_next_server()

    def add_server(self):
        self.servers_count +=1
        self.server_idle[self.servers_count] = True
        self.last_served_time[self.servers_count] = 0.0
        # print("added_server {0:.2f}".format(self.servers_count))

    def remove_server(self):
        if self.server_idle[self.servers_count]:
           self.server_idle.pop(self.servers_count)
           self.last_served_time.pop(self.servers_count)
           self.servers_count-=1
        # print("removed_server {0:.2f}".format(self.servers_count))



class ServiceEvent:
    def __init__(self, server):
        self.e_time = 0.0
        self.id = 0  # user Id
        self.server = server  # server Name
        self.service_time = 0

    def execute(self, devs):
        ind = [i for i, val in enumerate(devs.stats) if val.id == self.id][0]
        devs.stats[ind].service_ends = self.e_time
        devs.stats[ind].time_in_system = devs.stats[ind].service_ends - devs.stats[ind].arrival_time
        devs.stats[ind].waiting_time_in_queue = devs.stats[ind].service_begins - devs.stats[ind].arrival_time
        # devs.stats[ind].idle_time_of_server = devs.stats[ind].service_begins - devs.last_served_time[devs.stats[ind].server]
        queue_list = list(filter(lambda x: x.server == self.server or x.server =="none",devs.customer_queue))
        if len(queue_list) > 0:
            index = devs.customer_queue.index(queue_list[0])
            qid = devs.customer_queue.pop(index)
            qind = [i for i, val in enumerate(devs.stats) if val.id == qid.id][0]
            service = ServiceEvent(self.server)
            service_time = get_service_time(self.server)
            service.e_time = self.e_time + service_time
            service.id = qid.id
            devs.stats[qind].server = self.server
            devs.stats[qind].service_begins = self.e_time
            devs.stats[qind].service_time = service_time
            devs.events_queue.add_event(service)
        else:
            devs.server_idle[self.server] = True
        devs.last_served_time[self.server] = self.e_time


class ArrivalEvent:
    def __init__(self, m_a, exec_type):
        self.e_time = 0.0
        self.max_agents = m_a
        self.type = exec_type
    def execute(self, devs):
        customer = CustomerStat()
        customer.id = devs.newId
        customer.arrival_time = self.e_time
        if len(devs.stats) > 0:
            customer.inter_arrival_time = customer.arrival_time - devs.stats[-1].arrival_time

        server = None
        if self.type == SYSTEM_TYPE.PRE_SELECTED:
            server = devs.get_preselected_server()
        if self.type == SYSTEM_TYPE.SHORTEST_QUEUE:
            server = devs.get_shortest_queue_server()
        if self.type == SYSTEM_TYPE.GENERAL_QUEUE:
            server = devs.get_next_server()
        if self.type == SYSTEM_TYPE.DYNAMIC_SERVERS:
            server = devs.get_dynamic_server()

        if devs.newId < self.max_agents - 1:
            next_arrival = ArrivalEvent(self.max_agents, self.type)
            next_arrival.e_time = self.e_time + custom_arrival.rvs()
            devs.events_queue.add_event(next_arrival)
        if server:
            customer.server = server
        if server and devs.server_idle[server]:
            devs.server_idle[server] = False
            service = ServiceEvent(server)
            service_time = get_service_time(server)

            customer.service_time = service_time
            customer.service_begins = self.e_time
            service.e_time = self.e_time + service_time
            service.id = customer.id
            customer.server = server
            devs.events_queue.add_event(service)
        else:
            devs.customer_queue.append(customer)
        devs.newId = devs.newId + 1
        devs.stats.append(customer)

In [69]:
avg_time = [0,0,0,0]
prob_wait = [0,0,0,0]
avg_waiting_for = [0,0,0,0]
avg_time_spent = [0,0,0,0]
values = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]


def do_stuff(event,idx):
    devs = DEVS()
    devs.events_queue.add_event(event)
    while devs.events_queue.size > 0:
        devs.process_next_event()
    stats = devs.stats

    av_time_in_queue = sum([x.waiting_time_in_queue for x in stats]) / len(stats)
    values[idx][0] = values[idx][0] + av_time_in_queue
    print('Average waiting time: {0:.2f}'.format(av_time_in_queue))

    prob_to_wait = len([x for x in stats if x.waiting_time_in_queue > 0]) / len(stats)
    values[idx][1] = values[idx][1] + prob_to_wait
    print('Probability that a customer has to wait: {0:.2f}'.format(prob_to_wait))

    avTimeBetwArr = sum([x.inter_arrival_time for x in stats])/(len(stats)-1)
    print("Average time between arrivals: {0:.2f}".format(avTimeBetwArr))

    avServiceTime = sum([x.service_time for x in stats])/len(stats)
    print("Average service time: {0:.2f}".format(avServiceTime))

    numOfCustWhoWait = len([x for x in stats if x.waiting_time_in_queue>0])
    avTimeWhoWait = sum([x.waiting_time_in_queue for x in stats])/numOfCustWhoWait
    values[idx][2] = values[idx][2] + avTimeWhoWait
    print("Average waiting time for those who wait: {0:.2f}".format(avTimeWhoWait))

    avTimeInTheSystem1 = sum([x.time_in_system for x in stats])/len(stats)
    values[idx][3] = values[idx][3] + avTimeInTheSystem1
    print("Average time a customer spends in the system: {0:.2f}\n".format(avTimeInTheSystem1))

def toFixed(numObj, digits=2):
    return f"{numObj:.{digits}f}"

names = [""]
fields = ['Average waiting time','Probability that a customer has to wait','Average waiting time for those who wait','Average time a customer spends in the system']
for name, member in SYSTEM_TYPE.__members__.items():
    print(member.value)
    names.append(name)
    print('\n' + name + ': \n')
    # if name =="DYNAMIC_SERVERS":
    for x in range(3):
         do_stuff(ArrivalEvent(10000, member),member.value)
def dosmth(arr):
    arre = [toFixed(n/3) for n in arr]
    return arre

values = [dosmth(n) for n in values]


fig = go.Figure(data=[go.Table(header=dict(values=names),
                 cells=dict(values=[fields, values[0],values[1],values[2],values[3]]))
                     ])
fig.update_layout(width=800, height=800)
fig.show()

0

PRE_SELECTED: 

Average waiting time: 33.98
Probability that a customer has to wait: 0.91
Average time between arrivals: 2.26
Average service time: 6.29
Average waiting time for those who wait: 37.43
Average time a customer spends in the system: 40.27

Average waiting time: 27.64
Probability that a customer has to wait: 0.91
Average time between arrivals: 2.27
Average service time: 6.30
Average waiting time for those who wait: 30.46
Average time a customer spends in the system: 33.94

Average waiting time: 37.78
Probability that a customer has to wait: 0.92
Average time between arrivals: 2.24
Average service time: 6.30
Average waiting time for those who wait: 41.21
Average time a customer spends in the system: 44.08

1

SHORTEST_QUEUE: 

Average waiting time: 3.26
Probability that a customer has to wait: 0.62
Average time between arrivals: 2.26
Average service time: 6.30
Average waiting time for those who wait: 5.23
Average time a customer spends in the system: 9.56

Average waiting